In [ ]:
'''
测试1，是否GYM能用
'''

import gym
 
env = gym.make('CartPole-v0',render_mode=None)  # 获取环境
# env = gym.make('Pendulum-v0')
print(env.spec.id)  # env.spec.id = 'CartPole-v0'
#env.render()  # 把环境渲染出来
 
for i_episode in range(1):  # 20个序列
    observation = env.reset()  # 初始状态
    print(observation)
    for t in range(3):  # 每个序列的步长T
        #env.render()
        print('\n',t)
        action = env.action_space.sample()  # 动作空间随机采样动作
        observation, reward, done, info, _ = env.step(action)  # 环境采取动作后的反馈信息
        print(observation)
        print(type(observation))
        print(observation.dtype)
        print(info)
        print(i_episode, action, reward, observation)  # 打印信息
        
        print(env.observation_space.shape)
        print('观测空间 = {}'.format(env.observation_space))
        print(type(env.observation_space))
        print(env.observation_space.dtype)
 
        print(env.observation_space.high)
        print(type(env.observation_space.high))
        print(env.observation_space.high.shape)
 
        print(env.action_space.shape)
        print('动作空间 = {}'.format(env.action_space))
        print(type(env.action_space))
        print(env.action_space.dtype)
        if done:  # 若杆的倾斜度数过大，或者车子离初始位置左右的偏离程度过大，或者坚持时间到200帧，则游戏结束
            break
env.close()  # 结束环境

In [ ]:
'''
测试2，是否GYM能用,保存为GIF
'''

import gym
import imageio
import matplotlib.pyplot as plt
frames = []
def show():
    screen = env.render()
    plt.imshow(screen)
    plt.show()
    
env = gym.make('CartPole-v0',render_mode='rgb_array')  # 获取环境
# env = gym.make('Pendulum-v0')
print(env.spec.id)  # env.spec.id = 'CartPole-v0'

print('gym env running') 
for i_episode in range(1):  # 20个序列
    observation = env.reset()  # 初始状态
    for t in range(300):  # 每个序列的步长T
        frames.append(env.render())
        action = env.action_space.sample()  # 动作空间随机采样动作
        observation, reward, done, info, _ = env.step(action)  # 环境采取动作后的反馈信息
        #show()
        if done:  # 若杆的倾斜度数过大，或者车子离初始位置左右的偏离程度过大，或者坚持时间到200帧，则游戏结束
            break
print('gif writing')
with imageio.get_writer('/home/liuli/myCodes/testGYM/gym测试.gif', mode='I') as writer:
    for frame in frames:
        writer.append_data(frame)
env.close()  # 结束环境
print('ednded')

In [ ]:
'''
测试3，是否tensorflow-keras+gym能用
参考
https://blog.csdn.net/2401_84495872/article/details/139591541
'''

import gym # 导入Gym库，用于开发和比较强化学习算法
import numpy as np # 导入NumPy库，用于进行科学计算
import tensorflow as tf # 导入TensorFlow库
from tensorflow import keras # 导入keras模块，这是一个高级神经网络API
from tensorflow.keras import layers # 导入keras中的layers模块，用于创建神经网络层
import imageio


seed = 42 # 设定随机种子，用于复现实验结果
gamma = 0.99 # 定义折扣率，用于计算未来奖励的现值
max_steps_per_episode = 10000 # 设定每个 episode 的最大步数
env = gym.make("CartPole-v0",render_mode='rgb_array') # 创建 CartPole-v0 环境实例

eps = np.finfo(np.float32).eps.item() # 获取 float32 数据类型的误差最小值 epsilon 

#########################
num_inputs = 4 # 状态空间的维度，即输入层的节点数
num_actions = 2 # 行为空间的维度，即输出层的节点数
num_hidden = 128 # 隐藏层的节点数

inputs = layers.Input(shape=(num_inputs,)) # 创建输入层，指定输入的形状
common = layers.Dense(num_hidden, activation="relu")(inputs) # 创建一个全连接层，包含num_hidden 个神经元，使用 ReLU 作为激活函数
action = layers.Dense(num_actions, activation="softmax")(common) # 创建一个全连接层，包含 num_actions 个神经元，使用 softmax 作为激活函数
critic = layers.Dense(1)(common) # 创建一个全连接层，包含1个神经元

model = keras.Model(inputs=inputs, outputs=[action, critic]) # 创建一个 Keras 模型，包含输入层、共享的隐藏层和两个输出层


#########################

    
action_probs =np.array([0.5,0.5])    
env.reset() 
frames = []
for t in range(max_steps_per_episode):
    frames.append(env.render())
    observation, reward, done, info, _ = env.step(np.random.choice(num_actions, p=np.squeeze(action_probs)))
    if done:
        break

with imageio.get_writer('/home/liuli/myCodes/testGYM/未训练前的样子.gif', mode='I') as writer:
    for frame in frames:
        writer.append_data(frame)
        
        
        

#########################

optimizer = keras.optimizers.Adam(learning_rate=0.01) # 创建 Adam 优化器实例，设置学习率为 0.01
huber_loss = keras.losses.Huber() # 创建损失函数实例
action_probs_history = [] # 创建一个列表，用于保存 action 网络在每个步骤中采取各个行动的概率
critic_value_history = [] # 创建一个列表，用于保存 critic 网络在每个步骤中对应的值
rewards_history = [] # 创建一个列表，用于保存每个步骤的奖励值
running_reward = 0 # 初始化运行过程中的每轮奖励
episode_count = 0 # 初始化 episode 计数器



while True:  
    stateInit= env.reset()  # 新一轮游戏开始，重置环境
    state = stateInit[0]
    episode_reward = 0  # 记录本轮游戏的总奖励值
    with tf.GradientTape() as tape:  # 构建 GradientTape 用于计算梯度
        for timestep in range(1, max_steps_per_episode): # 本轮游戏如果一切正常会进行 max_steps_per_episode 步
            
            state = tf.convert_to_tensor(state)  # 将状态转换为张量
            
            state = tf.expand_dims(state, 0)  # 扩展维度，以适应模型的输入形状
            
            action_probs, critic_value = model(state)  # 前向传播，得到 action 网络输出的动作空间的概率分布，和 critic 网络预测的奖励值
            critic_value_history.append(critic_value[0, 0])  # 将上面 critic 预测的奖励值记录在 critic_value_history 列表中

            action = np.random.choice(num_actions, p=np.squeeze(action_probs))  # 依据概率分布抽样某个动作，当然了某个动作概率越大越容易被抽中，同时也保留了一定的随机性
            action_probs_history.append(tf.math.log(action_probs[0, action]))  # 将使用该动作的对数概率值记录在 action_probs_history 列表中
        
            state, reward, done, info, _ = env.step(action)  # 游戏环境使用选中的动作去执行，得到下一个游戏状态、奖励、是否终止和其他信息
            rewards_history.append(reward)  # 将该时刻的奖励记录在 rewards_history 列表中
            episode_reward += reward  # 累加本轮游戏的总奖励值

            if done:  # 如果到达终止状态，则结束循环
                break

        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward  # 计算平均奖励

        returns = []  # 存储折扣回报
        discounted_sum = 0
        for r in rewards_history[::-1]:  # 从后往前遍历奖励的历史值
            discounted_sum = r + gamma * discounted_sum  # 计算折扣回报
            returns.insert(0, discounted_sum)  # 将折扣回报插入列表的开头，最后形成的还是从前往后的折扣奖励列表

        returns = np.array(returns)  # 将折扣回报转换为数组
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)  # 归一化折扣回报
        returns = returns.tolist()  # 将折扣回报转换为列表形式

        history = zip(action_probs_history, critic_value_history, returns)  # 将三个列表进行 zip 压缩
        actor_losses = []  # 存储 action 网络的损失
        critic_losses = []  # 存储 critic 网络的损失

        for log_prob, value, ret in history:
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # 计算 actor 的损失函数

            critic_losses.append(
                huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0)) # 计算 critic 的损失函数
            )

        loss_value = sum(actor_losses) + sum(critic_losses) # 计算总损失函数
        grads = tape.gradient(loss_value, model.trainable_variables) # 计算梯度
        optimizer.apply_gradients(zip(grads, model.trainable_variables)) # 更新模型参数

        action_probs_history.clear() # 清空之前的历史记录
        critic_value_history.clear() # 清空之前的历史记录
        rewards_history.clear() # 清空之前的历史记录

    episode_count += 1 # 当一轮游戏结束时， episode 加一
    if episode_count % 10 == 0: # 每训练 10 个 episode ，输出当前的平均奖励
        template = "在第 {} 轮游戏中获得奖励: {:.2f} 分"
        print(template.format(episode_count, running_reward))

    if running_reward > 195:  # 如果平均奖励超过195，视为任务已经解决
        print("奖励超过 195 ，训练结束")
        break

        
###################

stateInit= env.reset()  # 新一轮游戏开始，重置环境
state = stateInit[0]
frames = []
for t in range(max_steps_per_episode):
    frames.append(env.render())
    
    state = tf.convert_to_tensor(state)  # 将状态转换为张量
    state = tf.expand_dims(state, 0)  # 扩展维度，以适应模型的输入形状
    
    action_probs, _ = model(state)
    action = np.random.choice(num_actions, p=np.squeeze(action_probs))
    state, reward, done,info, _ = env.step(action)
    if done:
        break

with imageio.get_writer('/home/liuli/myCodes/testGYM/训练后的样子.gif', mode='I') as writer:
    for frame in frames:
        writer.append_data(frame)
        
print('ended')





In [ ]:
'''
测试5，GYM+其他算法


gym: OpenAI提供的一个用于开发和比较强化学习算法的工具包，可以用于各种任务，包括但不限于环境交互、强化学习等。

TensorForce: 一个用于强化学习的库，它提供了基于深度强化学习的算法，并且可以与TensorFlow集成。

OpenAI Gym: 一个用于开发和比较强化学习算法的工具包，可以用于各种任务，包括但不限于环境交互、强化学习等。

Baselines: 一个提供各种强化学习算法的库，包括DQN、PPO、A2C等。

Dopamine: 一个用于研究复杂机器学习系统的开源库，主要关注强化学习。

RLlib: 用于分布式强化学习的工具包，可以在大型集群上训练A3C算法。

Coach: 由百度提供的，用于强化学习研究的开源框架。

强化学习算法库 stable_baselines3
https://zhuanlan.zhihu.com/p/652391991
'''



In [ ]:
'''

用TensorForce快速搭建深度强化学习模型
https://tensorforce.readthedocs.io/en/latest/basics/installation.html
1.conda create -n tensorforce python=3.8
2.conda activate tensorforce  
3.已经下载，并且必须执行，否则会出各种错误
git clone https://github.com/tensorforce/tensorforce.git (已经下载)
cd tensorforce
pip3 install -e . -i https://pypi.tuna.tsinghua.edu.cn/simple

4.conda install ipykernel 
5.ipython kernel install --user --name=tensorforce
'''




!python3 testTensorforce1.py #最简单测试
!python3 testTensorforce2.py #测试算法
!python3 testTensorforce3.py #看结果+GIF

In [ ]:
##测试自定义Veh2CrashEnv
!python3 Veh2CrashEnv.py
!python3 mainTestEnvVeh2.py #测试单独测试文件

#cd /root/miniconda3/envs/tensorforce/lib/python3.8/site-packages/gym/envs/my_env
!python3 mainTesTGymEnvVeh2.py #测试在GYM里面的VehEnv

In [ ]:
#测试4，是否tensorflow-keras+自定义gym-Veh2Crash+GPU(环境tensor23py36gpu)能用
!python testGymVeh2Gpu.py

In [ ]:
!cp /root/miniconda3/envs/tensorforce/lib/python3.8/site-packages/gym/envs/my_env/Veh2CrashEnv.py  /home/liuli/myCodes/testGYM/Veh2CrashEnv_tensorforce.py
!cp /root/miniconda3/envs/tensor23py36gpu/lib/python3.6/site-packages/gym/envs/my_env/Veh2CrashEnv.py  /home/liuli/myCodes/testGYM/Veh2CrashEnv_tensor23py36gpu.py

In [ ]:
!conda list 再tensor23py36gpu
udatoolkit               10.1.243            h8cb64d8_11    https://mirrors.ustc.edu.cn/anaconda/cloud/conda-forge
cudnn                     7.6.5.32             hc0a50b0_1    https://mirrors.ustc.edu.cn/anaconda/cloud/conda-forge
tensorflow-gpu            2.3.1                    pypi_0    pypi
keras                     2.3.1
gym                       0.26

In [ ]:
!!conda list 在tensorforce
tensorflow                2.11.1                   pypi_0    pypi
gym                       0.22

In [ ]:
'''
测试5，是否pytorch+自定义gym-Veh2Crash+GPU(环境torch24PY38Gpu)能用
'''
'''
建立环境torch24PY38Gpu
!conda create -n torch24PY38Gpu python=3.8
#!conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
!pip3 install torch torchvision torchaudio -i https://pypi.tuna.tsinghua.edu.cn/simple
!nvidia-smi
!pip install gymnasium==0.29
'''

import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())  #输出为True，则安装无误
!python test5_torchGymVeh2Gpu.py